<a href="https://colab.research.google.com/github/cedro3/others/blob/master/Facebook_VideoPose3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Install Pytorch & Caffe2**

In [ ]:
!wget https://anaconda.org/pytorch/pytorch/1.2.0/download/linux-64/pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!tar xvjf pytorch-1.2.0-py3.6_cuda10.0.130_cudnn7.6.2_0.tar.bz2
!cp -r lib/python3.6/site-packages/* /usr/local/lib/python3.6/dist-packages/

# check if Caffe2 was  build

In [ ]:
# To check if Caffe2 build was successful
!python -c 'from caffe2.python import core' 2>/dev/null && echo "Success" || echo "Failure"

# To check if Caffe2 GPU build was successful
!python -c 'from caffe2.python import workspace; print(workspace.NumCudaDevices())'

# Install COCO Dataset 

In [ ]:
!apt-get install python-dev
!pip install cython
!pip install pycocotools
!git clone https://github.com/cocodataset/cocoapi.git
!cd cocoapi/PythonAPI && make install

import os
os.environ['COCOAPI'] = ":/content/cocoapi"

# Install Detectron

In [ ]:
!git clone https://github.com/facebookresearch/detectron
!pip install -r detectron/requirements.txt
!cd detectron && make
!python detectron/detectron/tests/test_spatial_narrow_as_op.py

# Install VideoPose3D & Copy Video Script to Detectron Tools Folder

In [ ]:
#copy file from VideoPose3d
!git clone https://github.com/facebookresearch/VideoPose3D
!cp VideoPose3D/inference/infer_video.py detectron/tools/infer_video.py

# Download Pretrained Human3.6m Coco Model

In [ ]:
!mkdir VideoPose3D/checkpoint
os.chdir('VideoPose3D/checkpoint')
!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_detectron_coco.bin
os.chdir('../..')

# Download Youtube Video for 3D Pose Estimation (specify YOUTUBE_ID)

In [ ]:
YOUTUBE_ID ='cgHZJiyWKIY'


!pip install -q youtube-dl
#download video
!youtube-dl -f 'bestvideo[ext=mp4]' --output "youtube.%(ext)s" https://www.youtube.com/watch?v=$YOUTUBE_ID

!mkdir videos   
  
# cut the first 5 seconds
!ffmpeg -y -loglevel info -i youtube.mp4 -ss 00:00:48 -t 00:00:14 videos/video.mp4

# recommendet alternation to 50fps 
#!ffmpeg -i videos/video.mp4 -filter "minterpolate='fps=50'" -crf 0 videos/video50fps.mp4

# Compute 2D Coordinates with Detectron

In [ ]:
ls

In [ ]:
!mkdir output
!python detectron/tools/infer_video.py \
    --cfg detectron/configs/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml \
    --output-dir output \
    --image-ext mp4 \
    --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \
   videos
    
#	  --wts https://dl.fbaipublicfiles.com/detectron/37698009/12_2017_baselines/e2e_keypoint_rcnn_R-101-FPN_s1x.yaml.08_45_57.YkrJgP6O/output/train/keypoints_coco_2014_train:keypoints_coco_2014_valminusminival/generalized_rcnn/model_final.pkl \

    

# Prepare Detectron Output to fit VideoPose3D Input

In [ ]:

!mkdir ./VideoPose3D/data/detectronoutput
!cp output/video.mp4.npz VideoPose3D/data/detectronoutput/video.mp4.npz
os.chdir('VideoPose3D/data') # This script must be launched from the "data" directory
!python prepare_data_2d_custom.py -i detectronoutput -o myvideos
os.chdir('../../')

# Compute 3D Joints with VideoPose3D

In [ ]:
#os.chdir('../')

#os.chdir('checkpoint')
#!wget https://dl.fbaipublicfiles.com/video-pose-3d/pretrained_h36m_cpn.bin
##!wget https://dl.fbaipublicfiles.com/video-pose-3d/d-pt-243.bin

!cp ./videos/video.mp4 VideoPose3D/video.mp4
os.chdir('VideoPose3D')


#!python run.py -d custom -k MyCustomDatasetName -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject S1 --viz-action custom --viz-camera 0 --viz-export My3dDataExport --viz-size 6
#!python run.py -d custom -k MyCustomDatasetName -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject  video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-size 6

#!python run.py -e 80 -k gt -arc 3,3,3,3,3


#!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --viz-export My3dDataExport

#!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate d-pt-243.bin --render --viz-subject video.mp4 --viz-action Directions --viz-video video.mp4 --viz-camera 0 --viz-output output_scater.mp4 --viz-size 5 --viz-downsample 1 --viz-skip 9

#!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-size 6
!python run.py -d custom -k myvideos -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_detectron_coco.bin --render --viz-subject video.mp4 --viz-action custom --viz-camera 0 --viz-video video.mp4 --viz-output output.mp4 --viz-export outputfile --viz-size 6

#working version 
#!python run.py -k gt -arc 3,3,3,3,3 -c checkpoint --evaluate pretrained_h36m_cpn.bin --viz-export My3dDataExport --viz-output output.mp4

# Display Results - Joint Export 

In [ ]:
#inspect joints export 

import numpy as np
data  = np.load('outputfile.npy')
lst = data
for item in lst:
    print(item)
    


# Display Results - Joint Video

In [ ]:
#display video
def show_local_mp4_video(file_name, width=640, height=480):
  import io
  import base64
  from IPython.display import HTML
  video_encoded = base64.b64encode(io.open(file_name, 'rb').read())
  return HTML(data='''<video width="{0}" height="{1}" alt="test" controls>
                        <source src="data:video/mp4;base64,{2}" type="video/mp4" />
                      </video>'''.format(width, height, video_encoded.decode('ascii')))

show_local_mp4_video('output.mp4', width=960, height=720)

#Download Joint Export & Video

In [ ]:
from google.colab import files


files.download('output.mp4')
files.download('outputfile.npy')